<a href="https://colab.research.google.com/github/sai8e/StockMarketProject/blob/main/Stock%20Predictor%20using%20fbprophet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fbprophet
!pip install plotly
!pip install streamlit
!pip install yfinance


In [59]:
%%writefile stock.py
import streamlit as st
from datetime import date
import yfinance as yf
from fbprophet import Prophet
from fbprophet.plot import plot_plotly
from plotly import graph_objs as go 

Start = "2015-01-01"
Today = date.today().strftime("%Y-%m-%d")

st.title("Stock Prediction App")


stock = st.text_input("Choose a Stock to Predict", "TSLA")
#stocks = ("AAPL", "GOOG", "MSFT", "GME")
#selected_stock = st.selectbox("Select dataset for prediction", stocks)

n_days = st.slider("Weeks to predict:",1 , 5)
period = n_days *365


@st.cache
def load_data(ticker):
    data = yf.download(ticker, Start, Today)
    data.reset_index(inplace=True)
    return data

data_load_state = st.text("Load data...")
data = load_data(stock)
data_load_state.text("Loading data...Done")

st.subheader('Raw Data')
st.write(data.tail())

def plot_raw_data():
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=data['Date'],y = data['Open'],name = 'stock_open'))
  fig.add_trace(go.Scatter(x=data['Date'],y = data['Close'],name = 'stock_close'))
  fig.layout.update(title_text='Time serious Data', xaxis_rangeslider_visible=True)
  st.plotly_chart(fig)

plot_raw_data()

# Forecast
df_train = data[['Date','Close']]
df_train = df_train.rename(columns={"Date": "ds", "Close": "y"})

m = Prophet()
m.fit(df_train)
future = m.make_future_dataframe(periods=period)
forecast = m.predict(future)

st.subheader('Forecast data')
st.write(forecast.tail())


st.write("forecast data")
fig1 = plot_plotly(m, forecast)
st.plotly_chart(fig1)

st.write('Forecast Component')
fig2 = m.plot_components(forecast)
st.write(fig2)






Writing stock.py


In [60]:
!streamlit run stock.py & npx localtunnel --port 8501


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.71.216.37:8501

npx: installed 22 in 3.062s
your url is: https://pink-robin-93.loca.lt
2021-11-15 21:47:02.616 NumExpr defaulting to 2 threads.
[*********************100%***********************]  1 of 1 completed
2021-11-15 21:47:04.664 Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
Initial log joint probability = -101.517
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       4476.79     0.0087782       186.453           1           1      116   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       4508.42    0.00208422       342.169           1           1      231   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       4514.11    0.00147382       291.948      0.7434  